# Predicción de demanda de alquileres sitycleta por países

In [7]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

In [8]:
df_clima = pd.read_csv('data/time_df.csv', sep=';')
df_clima

,prec,presMax,presMin,racha,tmax,tmed,tmin,velmedia,moth,day
0,3.1,1015.7,1012.0,11.4,22.3,20.6,19.0,2.8,12,1
1,1.1,1012.9,1008.8,7.8,23.0,20.7,18.4,3.3,12,2
2,7.1,1009.7,1006.5,6.7,20.1,18.9,17.7,1.7,12,3
3,0.0,1007.7,1004.9,3.9,23.0,20.1,17.2,1.1,12,4
4,0.4,1015.0,1006.1,6.9,24.2,22.0,19.9,1.4,12,5
5,0.0,1015.1,1010.9,3.6,24.9,21.7,18.5,0.3,12,6
6,0.8,1010.9,1005.6,3.3,22.7,20.8,19.0,1.1,12,7
7,0.0,1010.0,1005.7,4.7,23.7,22.0,20.2,0.6,12,8
8,0.0,1015.2,1009.2,6.1,25.0,22.4,19.9,0.8,12,9
9,0.0,1017.3,1014.1,5.0,24.8,22.4,20.0,0.8,12,10


In [15]:
df_sagulpa = pd.read_csv('data/sagulpa_df.csv', sep=';')
df_sagulpa

,id_country,country,start_place,start_day,start_month,start_h,start_min,start_seg,end_day,end_month,end_h,end_min,end_seg,user_time
0,18,Polonia,Plazoleta de Farray,13,12,20,59,23,14,12,8,6,9,11:06:46
1,20,Hungría,C.C. El Muelle,2,12,12,51,2,2,12,21,54,10,09:03:08
2,14,Finlandia,Plaza de España (elect.),27,12,11,12,16,27,12,19,50,51,08:38:35
3,11,Reino Unido,Teatro Pérez Galdós,27,12,22,18,4,28,12,6,37,52,08:19:48
4,11,Reino Unido,Churruca,30,12,22,29,17,31,12,6,47,42,08:18:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,18,Polonia,San Telmo,28,12,14,37,34,28,12,14,38,28,00:00:54
4280,15,Italia,Plaza de la feria,13,12,11,46,49,13,12,11,47,38,00:00:49
4281,11,Reino Unido,Plazoleta de Farray,4,12,21,8,3,4,12,21,8,49,00:00:46
4282,14,Finlandia,Plaza de la feria,1,12,9,11,42,1,12,9,12,27,00:00:45


In [16]:
df_sagulpa[['user_h', 'user_min', 'user_seg']] = df_sagulpa['user_time'].str.split(':', expand=True)
df = df.drop('user_time', axis=1)
df

,id_country,country,start_place,start_day,start_month,start_h,start_min,start_seg,end_day,end_month,end_h,end_min,end_seg,user_h,user_min,user_seg
0,18,Polonia,Plazoleta de Farray,13,12,20,59,23,14,12,8,6,9,11,06,46
1,20,Hungría,C.C. El Muelle,2,12,12,51,2,2,12,21,54,10,09,03,08
2,14,Finlandia,Plaza de España (elect.),27,12,11,12,16,27,12,19,50,51,08,38,35
3,11,Reino Unido,Teatro Pérez Galdós,27,12,22,18,4,28,12,6,37,52,08,19,48
4,11,Reino Unido,Churruca,30,12,22,29,17,31,12,6,47,42,08,18,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,18,Polonia,San Telmo,28,12,14,37,34,28,12,14,38,28,00,00,54
4280,15,Italia,Plaza de la feria,13,12,11,46,49,13,12,11,47,38,00,00,49
4281,11,Reino Unido,Plazoleta de Farray,4,12,21,8,3,4,12,21,8,49,00,00,46
4282,14,Finlandia,Plaza de la feria,1,12,9,11,42,1,12,9,12,27,00,00,45


## Regresion por paices saber cantidad de alquiler por paices
## tranformar las estaciones de salida a numerico

In [18]:
df_clima.columns

Index(['prec', 'presMax', 'presMin', 'racha', 'tmax', ' tmed', ' tmin',
       ' velmedia', 'moth', 'day'],
      dtype='object')

In [17]:
for index, row in df_sagulpa.iterrows():
    match = df_clima[(df_clima['day'] == row['start_day'])]
    if not match.empty:
        df_sagulpa.loc[index, 'prec'] = match.iloc[0]['prec']
        df_sagulpa.loc[index, 'presMax'] = match.iloc[0]['presMax']
        df_sagulpa.loc[index, 'presMin'] = match.iloc[0]['presMin']
        df_sagulpa.loc[index, 'tmax'] = match.iloc[0]['tmax']
        df_sagulpa.loc[index, 'tmed'] = match.iloc[0]['tmed']
        df_sagulpa.loc[index, 'tmin'] = match.iloc[0]['tmin']
        df_sagulpa.loc[index, 'velmedia'] = match.iloc[0]['velmedia']

KeyError: 'tmed'

In [58]:
df_sagulpa.to_csv('sagulpa.csv', index=False)